# GEOtop Calibration Report

In [ ]:
# Default

model_path = None
observations_path = None
parameters_path = 'data/parameters/default.csv'
timeout = 300
default_parameters = {'FirstLayerWidth': 10, 'MaxDepth': 1000}
algorithm = 'NGO'
budget = None
popsize = None
num_workers = None
scheduler_file = None

In [ ]:
from time import strftime, gmtime

import numpy as np
import pandas as pd
import scrapbook as sb
from dask.distributed import Client

from mhpc_project.utils import date_parser, comparison_plots, convergence_plot, time_loss_plot, calibrate
from mhpc_project.parameters import VarSoilParameters
from mhpc_project.models import VarSoilFullModel

In [ ]:
# Glue inputs
sb.glue('model_path', model_path)
sb.glue('observations_path', observations_path)
sb.glue('parameters_path', parameters_path)
sb.glue('timeout', timeout)
sb.glue('default_parameters', default_parameters)
sb.glue('algorithm', algorithm)
sb.glue('budget', budget)
sb.glue('num_workers', num_workers)

In [ ]:
parameters = VarSoilParameters(parameters_path, default_parameters)
model = VarSoilFullModel(model_path, store=False, timeout=timeout)
observations = pd.read_csv(observations_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)

In [ ]:
client = Client(scheduler_file=scheduler_file)

In [ ]:
comparison_plots(model, observations, parameters.instrumentation)

In [ ]:
recommendation, log, elapsed = calibrate(model,
                                         parameters,
                                         observations,
                                         budget,
                                         algorithm,
                                         popsize,
                                         client,
                                         num_workers)
print("elapsed time:", strftime("%T", gmtime(elapsed)))

In [ ]:
convergence_plot(log)

In [ ]:
time_loss_plot(log)

In [ ]:
comparison_plots(model, observations, recommendation)

In [ ]:
parameters_best = parameters.from_instrumentation(recommendation, column_name='best')
report = parameters.delta_mim(log)
report['best'] = parameters_best
report.sort_values('delta', key=np.abs, ascending=False)

In [ ]:
# Outputs
sb.glue('report', report, 'pandas')
sb.glue('loss', recommendation.loss)
sb.glue('elapsed_time', elapsed)
